In [ ]:
from netmiko import SSHDetect, ConnectHandler
from getpass import getpass
import json

In [ ]:
password = getpass()

In [ ]:
#deviceType = "cisco_ios"	# Optional
deviceType = "autodetect"
sessionlogfile="log/sessionlogging.log"
host = "192.168.0.1"
username = "myusername"

equip = {
    "device_type":deviceType, 
	"ip":host, 
	"username":username,
	"password":password,
    "session_log":sessionlogfile
}

In [ ]:
device = ConnectHandler(**equip)

In [ ]:
cmd = 'show version'
s = device.send_command(cmd)
stop = False

eq_vendor = ''
eq_model = ''

if s.upper().find('CISCO') > -1:
    eq_vendor = 'Cisco'
else:
    stop = True
        

In [ ]:
print(eq_vendor)
print(eq_model)

In [ ]:
if eq_vendor == 'Cisco':
    commands2 = [
        'show version',
        'show inventory',
        'show running config',
        'show ip int brief',
        'show interface'
    ]
    
dicts_output = {}
dicts_output["vendor"] = eq_vendor
dicts_output["model"] = eq_model

outputA = []
for cmd in commands2:
    s = device.send_command(cmd)
    outputA.append(s)
    dicts_output[cmd] = s

In [ ]:
# for op in outputA:
#     print(op)

In [ ]:
json_string = json.dumps(dicts_output)
# print(json_string)
json_formatted_str = json.dumps(dicts_output, indent=4)
# print(json_formatted_str)

In [ ]:
output = device.send_command('show ip route')
print (output)

In [ ]:
output = device.send_command("show ip route", use_genie=True)
print(json.dumps(output, indent=4, sort_keys=True))

In [ ]:
device.disconnect()

In [ ]:
from pymongo import MongoClient

In [ ]:
myclient = MongoClient("mongodb://localhost:27017/")
db = myclient["Something"]
Collection = db["datastuff"]

In [ ]:
if isinstance(json_formatted_str, list):
    Collection.insert_many(json_formatted_str) 
else:
    Collection.insert_one(json_formatted_str)